In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 0. [필수] frames_none과 frames_hu 데이터프레임 로드 ---
# 이 부분에 실제 온도 데이터 로드 코드를 작성하세요.
# 예시: 가상의 온도 데이터프레임을 생성 (실제 데이터 대신)
# 실제 데이터가 있다면 아래 가상 데이터 생성 코드는 제거하세요.
np.random.seed(42) # 재현성을 위해 시드 설정

# frames_none (4073, 64) - 배경만 있는 데이터
# 예시: 평균 25도, 표준편차 1도의 노이즈
data_none_temps = np.random.uniform(24.0, 26.0, size=(4073, 8*8))
frames_none = pd.DataFrame(data_none_temps)

# frames_hu (2000, 64) - 열원 포함 데이터
# 예시: 배경 온도 + 열원 효과 (특정 영역에 높은 온도 값)
data_hu_temps = np.random.uniform(24.0, 26.0, size=(2000, 8*8))
frames_hu = pd.DataFrame(data_hu_temps)

frame_height, frame_width = 8, 8 # 이미지 크기
heat_source_size = 3 # 열원 크기 (시뮬레이션 용도)
heat_source_temp_boost = 15.0 # 열원 영역의 온도 상승량 (시뮬레이션 용도)

# frames_hu에 '이미' 열원이 존재하도록 시뮬레이션
# 실제 데이터를 사용할 경우 아래 for 루프를 삭제하세요.
# for i in range(len(frames_hu)):
#     if i % 2 == 0: # 대략 절반의 프레임에만 열원 추가 (예시)
#         frame_2d_temp = frames_hu.iloc[i].values.reshape(frame_height, frame_width)
        
#         # 고정된 위치에 열원 추가 (예시)
#         x_start = (i * 1) % (frame_width - heat_source_size + 1)
#         y_start = (i * 2) % (frame_height - heat_source_size + 1)
        
#         frame_2d_temp[y_start:y_start+heat_source_size, x_start:x_start+heat_source_size] += heat_source_temp_boost
        
#         frames_hu.iloc[i] = frame_2d_temp.flatten()

# print(f"frames_none (배경 온도 프레임) 로드/생성 완료. Shape: {frames_none.shape}")
# print(f"frames_hu (열원 포함 온도 프레임) 로드/생성 완료. Shape: {frames_hu.shape}")
# print("-" * 30)

# --- 온도 데이터를 0-255 범위로 스케일링하는 함수 ---
def scale_temperature_to_uint8(temp_data, min_val, max_val):
    """
    온도 데이터를 0-255 범위의 uint8로 스케일링합니다.
    min_val: 전체 데이터에서 예상되는 최소 온도 값
    max_val: 전체 데이터에서 예상되는 최대 온도 값
    """
    scaled_data = (temp_data - min_val) / (max_val - min_val) * 255
    scaled_data = np.clip(scaled_data, 0, 255) # 0-255 범위를 벗어나지 않도록 클리핑
    return scaled_data.astype(np.uint8)

# 전체 온도 데이터의 예상 최소/최대값 설정 (데이터 특성에 맞게 조정 필요)
# 이 값을 기반으로 스케일링이 이루어집니다.
global_min_temp = min(frames_none.min().min(), frames_hu.min().min())
global_max_temp = max(frames_none.max().max(), frames_hu.max().max()) # 오타 수정
print(f"온도 데이터 전체 범위: {global_min_temp:.2f}°C ~ {global_max_temp:.2f}°C")
print("-" * 30)
 
# --- 1. 데이터프레임 할당 ---
frames_background = frames_none
frames_with_object = frames_hu

print(f"배경 프레임 수: {len(frames_background)}")
print(f"열원 포함 프레임 수: {len(frames_with_object)}")
print("-" * 30)

# 2. MOG2 배경 제거기 생성
fgbg = cv2.createBackgroundSubtractorMOG2(history=len(frames_background), varThreshold=16, detectShadows=False)

# --- 가우시안 필터 파라미터 ---
# ksize: 가우시안 커널 크기 (양의 홀수여야 함)
# sigmaX: X축 방향 표준 편차 (0이면 ksize에 따라 자동 계산)
gaussian_ksize = (3, 3) # 작은 커널로 미세 노이즈 제거
gaussian_sigmaX = 0    # 자동 계산

# 3. 배경 프레임으로 배경 모델 초기 학습
print("--- 배경 프레임으로 배경 모델 학습 시작 ---")
for i in range(len(frames_background.index)):
    temp_frame_2d = frames_background.iloc[i,:].values.reshape((frame_height, frame_width))
    scaled_frame = scale_temperature_to_uint8(temp_frame_2d, global_min_temp, global_max_temp)
    
    # 가우시안 필터 적용
    blurred_frame = cv2.GaussianBlur(scaled_frame, gaussian_ksize, gaussian_sigmaX)

    fgmask_dummy = fgbg.apply(blurred_frame) # 필터링된 프레임으로 학습
    
    if (i + 1) % 500 == 0:
        print(f"  {i + 1}/{len(frames_background)} 프레임 학습 완료...")
print("--- 배경 모델 학습 완료 ---")
print("-" * 30)

# 4. 객체가 있는 프레임을 넣어 전경 감지 및 추출
original_frames_results = [] # 스케일링 전 원본 (시각화용)
blurred_frames_results = [] # 가우시안 필터 적용 후 프레임
fg_masks_results = []
fg_only_frames_results = []

print("--- 열원 포함 프레임 처리 시작 ---")
for i in range(len(frames_with_object.index)):
    temp_frame_2d = frames_with_object.iloc[i,:].values.reshape((frame_height, frame_width))
    scaled_frame = scale_temperature_to_uint8(temp_frame_2d, global_min_temp, global_max_temp)
    
    # 가우시안 필터 적용
    blurred_frame = cv2.GaussianBlur(scaled_frame, gaussian_ksize, gaussian_sigmaX)

    fgmask = fgbg.apply(blurred_frame) # 필터링된 프레임으로 전경 감지
    
    # 전경 마스크를 사용하여 스케일링된 원본 프레임에서 열원만 추출 (필터링되지 않은 원본에 마스크 적용)
    # 또는 필터링된 프레임에 마스크를 적용할 수도 있습니다. 여기서는 필터링된 프레임에 적용합니다.
    fg_only = cv2.bitwise_and(blurred_frame, blurred_frame, mask=fgmask)

    original_frames_results.append(scaled_frame) 
    blurred_frames_results.append(blurred_frame) # 가우시안 적용된 프레임 추가
    fg_masks_results.append(fgmask)
    fg_only_frames_results.append(fg_only)

    if (i + 1) % 200 == 0:
        print(f"  {i + 1}/{len(frames_with_object)} 프레임 처리 완료...")
print("--- 열원 포함 프레임 처리 완료 ---")
print("-" * 30)


# 5. 결과 시각화
def display_images_temp(img1, title1, img2, title2, cmap_val='hot'):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img1, cmap=cmap_val) 
    plt.title(title1)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img2, cmap=cmap_val) 
    plt.title(title2)
    plt.axis('off')
    plt.show()

def display_full_pipeline(original_img, blurred_img, fg_mask, fg_only_img, idx):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5)) # 4개 서브플롯
    
    axes[0].imshow(original_img, cmap='hot')
    axes[0].set_title(f'Original Thermal {idx}')
    axes[0].axis('off')

    axes[1].imshow(blurred_img, cmap='hot')
    axes[1].set_title(f'Gaussian Blurred {idx}')
    axes[1].axis('off')

    axes[2].imshow(fg_mask, cmap='gray')
    axes[2].set_title(f'FG Mask {idx}')
    axes[2].axis('off')

    axes[3].imshow(fg_only_img, cmap='hot')
    axes[3].set_title(f'Heat Source Only {idx}')
    axes[3].axis('off')
    plt.tight_layout()
    plt.show()


# 시각화할 프레임 인덱스 선택 (요청하신 대로 확장)
display_idx = [
    0,                                      
    min(500, len(frames_with_object) - 1),  
    min(750, len(frames_with_object) - 1),  
    min(1000, len(frames_with_object) - 1), 
    min(1250, len(frames_with_object) - 1), 
    min(1500, len(frames_with_object) - 1), 
    min(1750, len(frames_with_object) - 1), 
]

# 각 인덱스에 대해 파이프라인 시각화
for idx in display_idx:
    display_full_pipeline(
        original_frames_results[idx],
        blurred_frames_results[idx],
        fg_masks_results[idx],
        fg_only_frames_results[idx],
        idx
    )

# 6. 마스크 후처리 (선택 사항)
post_process_idx = min(100, len(fg_masks_results) - 1) 
fgmask_example = fg_masks_results[post_process_idx]
original_frame_for_post = original_frames_results[post_process_idx] # 후처리에 사용할 원본 프레임

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

dilated_mask = cv2.dilate(fgmask_example, kernel, iterations=1)
eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

# 후처리된 마스크를 필터링된 프레임에 적용하여 전경 추출
# 만약 마스크를 원본 스케일링된 프레임에 적용하고 싶다면 original_frame_for_post를 사용
fg_only_processed = cv2.bitwise_and(blurred_frames_results[post_process_idx], blurred_frames_results[post_process_idx], mask=eroded_mask)

print(f"\n--- 마스크 후처리 결과 (열원 포함 프레임 {post_process_idx}) ---")
display_images_temp(fgmask_example, 'Original FG Mask', eroded_mask, 'Processed FG Mask', cmap_val='gray')
display_images_temp(original_frame_for_post, 'Original Thermal Frame', fg_only_processed, 'Processed Heat Source Only', cmap_val='hot')